In [1]:
import pandas as pd
import numpy as np
import torch
import io
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
import time
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report


In [2]:
%ls
prefix = '/data1/tjss/'
# prefix = '/content/drive/My Drive/Summer research /'
doc_level_df = pd.read_csv(prefix+'data/msgs_tec.csv')
word_level_df = pd.read_csv(prefix+'data/NRC_ht_emotion_sentiment.csv')

MeanEmbeddingsFasttext_NRCJoyVsNotJoy_wordLevelBackground(All_Reviews)Shap.ipynb
MeanEmbeddingsFasttext_NRCJoyVsNotJoy_wordLevelBackground(Joy_Reviews_ALL)Shap.ipynb
MeanEmbeddingsFasttext_NRCJoyVsNotJoy_wordLevelBackground(Joy_Reviews)Shap.ipynb
MeanEmbeddingsFasttext_NRCJoyVsNotJoy_wordLevelBackground(Non_Joy_Reviews_ALL)Shap.ipynb
MeanEmbeddingsFasttext_NRCJoyVsNotJoy_wordLevelBackground(Non_Joy_Reviews)Shap.ipynb
SummedEmbeddingsFasttext_NRCJoyVsNotJoy_wordLevelBackground(All_Reviews)Shap.ipynb
SummedEmbeddingsFasttext_NRCJoyVsNotJoy_wordLevelBackground(Joy_Reviews_ALL)Shap.ipynb
SummedEmbeddingsFasttext_NRCJoyVsNotJoy_wordLevelBackground(Joy_Reviews)Shap.ipynb
SummedEmbeddingsFasttext_NRCJoyVsNotJoy_wordLevelBackground(Non_Joy_Reviews)Shap.ipynb


/home/tjss/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#EDA
print(doc_level_df.shape)
print(doc_level_df.emotion.unique())
doc_level_df = doc_level_df.dropna()
print(doc_level_df.shape)
doc_level_df['message'] = doc_level_df['message'].str.lower() 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
lb_make = LabelEncoder()
doc_level_df['emotion_code'] = lb_make.fit_transform(doc_level_df['emotion'])
print(doc_level_df.iloc[:10,:])
new_doc_level_df = doc_level_df[['message', 'emotion_code']]
print(new_doc_level_df.iloc[:10,:])
new_doc_level_df.head(10)

new_doc_level_df.loc[(new_doc_level_df.emotion_code!=3),'emotion_code']=0
new_doc_level_df.loc[(new_doc_level_df.emotion_code==3),'emotion_code']=1

new_doc_level_train, new_doc_level_test = train_test_split(new_doc_level_df, test_size=0.2, random_state=1)
new_doc_level_train, new_doc_level_val = train_test_split(new_doc_level_train, test_size=0.25, random_state=1)

(21051, 3)
['surprise' 'sadness' 'joy' 'disgust' 'fear' 'anger' nan]
(21049, 3)
           message_id                                            message  \
0  145353048817012736  thinks that @melbahughes had a great 50th birt...   
1  144279638024257536  como una expresiã³n tan simple, una sola oraci...   
2  140499585285111809  the moment when you get another follower and y...   
3  145207578270507009  be the greatest dancer of your life! practice ...   
4  139502146390470656  eww.. my moms starting to make her annual rum ...   
5  146042696899887106  if ur heart hurts all the time for tht person ...   
6  145492569609084928  i feel awful, and it's way too freaking early....   
7  145903955229151232  so chuffed for safc fans! bet me dar comes in ...   
8  142717613234069504  making art and viewing art are different at th...   
9  144183822873927680  soooo dooowwwn!! move on, get some sleep... me...   

    emotion  emotion_code  
0  surprise             5  
1   sadness             4  

/home/tjss/miniconda3/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [4]:
print(new_doc_level_train.emotion_code.value_counts())
print(new_doc_level_val.emotion_code.value_counts())
print(new_doc_level_test.emotion_code.value_counts())

0    7706
1    4923
Name: emotion_code, dtype: int64
0    2570
1    1640
Name: emotion_code, dtype: int64
0    2534
1    1676
Name: emotion_code, dtype: int64


In [5]:
print(new_doc_level_train.shape)
print(new_doc_level_val.shape)
print(new_doc_level_test.shape)

(12629, 2)
(4210, 2)
(4210, 2)


In [6]:
# 0-> not Joy
# 1-> Joy


# 0 Anger
# 1 Disgust
# 2 Fear
# 3 Joy
# 4 Sadness
# 5 Surprise

In [7]:
new_doc_level_train.to_csv(prefix+"data/new_doc_level_train.csv")
new_doc_level_val.to_csv(prefix+"data/new_doc_level_val.csv")
new_doc_level_test.to_csv(prefix+"data/new_doc_level_test.csv")

In [8]:
def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = map(float, tokens[1:])
    return data


In [9]:
embeddings = load_vectors(prefix+"data/crawl-300d-2M.vec")

In [10]:
print(type(embeddings))
print(len(embeddings))
for word in embeddings:
  embeddings[word] = [float(i) for i in embeddings[word]]
embeddings_keylist = list(embeddings)
embeddings_vectorlist = list(embeddings.values())
word_to_index={}
index_to_word={}
for i in range(len(embeddings_keylist)):
  word_to_index[embeddings_keylist[i]]=i
  index_to_word[i]=embeddings_keylist[i]

<class 'dict'>
1999995


In [11]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

# from spacy.lang.en import English
# nlp = English()
# Create a Tokenizer with the default settings for English
# including punctuation rules and exceptions
# tokenizer = nlp.Defaults.create_tokenizer(nlp)



[nltk_data] Downloading package punkt to /home/tjss/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
# # Dummy dataloader
# test_df = pd.read_csv(prefix+"data/new_doc_level_val.csv")
# print(test_df.shape)
# test_df = test_df.iloc[:,1:]
# curr_tex = test_df.iloc[6,0]
# curr_emot = test_df.iloc[6,1]
# curr_embed = np.zeros((300))
# print("curr text", curr_tex)
# print(curr_embed)
# tot_cnt=0
# ign_cnt=0
# for word in word_tokenize(curr_tex):
#   print(word)
#   tot_cnt+=1
#   if word in embeddings.keys():
#     curr_embed = curr_embed+embeddings[word]
#   else:
#     ign_cnt+=1
# print("percentage lost", ign_cnt/tot_cnt*100, "%")
# print(curr_embed.shape)
# curr_embed = curr_embed/tot_cnt
# embed_t = torch.from_numpy(curr_embed)
# print(embed_t)

In [13]:
# device = "cuda" if torch.cuda.is_available() else "cpu"

# kwargs = {'num_workers': 1, 'pin_memory': True} if device=='cuda' else {}

In [14]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

In [15]:
class FastTextVectorLoader(Dataset):
  def __init__(self, csv_file, embeddings):
    self.frame = pd.read_csv(csv_file)
    self.embeddings = embeddings
  def __len__(self):
    return len(self.frame)
  def __getitem__(self, idx):
    if torch.is_tensor(idx):
      idx = idx.tolist()
    curr_tex = self.frame.iloc[idx,1]
    curr_emot = self.frame.iloc[idx,2]
    curr_embed = np.zeros((300))
    tot_cnt=0
    ign_cnt=0
    
    for word in word_tokenize(curr_tex):
      tot_cnt+=1
      if word in embeddings.keys():
        # print(word)
        curr_embed = curr_embed+embeddings[word]
      else:
        ign_cnt+=1
    # print("percentage lost", ign_cnt/tot_cnt*100, "%")
    curr_embed = curr_embed/tot_cnt
    embed_t = torch.from_numpy(curr_embed)
    # embed_t = embed_t.to(torch.device("cuda:0"))
    # curr_emot = curr_emot.cuda()
    data = {'vector':embed_t, 'label':curr_emot}
    return data

In [16]:
train_dataset = FastTextVectorLoader(prefix+"data/new_doc_level_train.csv", embeddings)
val_dataset = FastTextVectorLoader(prefix+"data/new_doc_level_val.csv", embeddings)
test_dataset = FastTextVectorLoader(prefix+"data/new_doc_level_test.csv", embeddings)

In [17]:
train_dataloader = DataLoader(train_dataset, batch_size=64,shuffle=True, num_workers=1)
val_dataloader = DataLoader(val_dataset, batch_size=64,shuffle=True, num_workers=1)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=1)


In [18]:
# testing dataloader
count = 0
for i,batch_data in enumerate(train_dataloader):
  count += 1
  print(batch_data['vector'])
  print(batch_data['vector'].size())
  print(batch_data['label'])
  # batch_data['vector'].cuda()
  # batch_data['vector'].to(device)
  print(batch_data['vector'].is_cuda)
  print(batch_data['label'].is_cuda)
  # print(text)
  # print(text_length)
  if count == 1:
    break


tensor([[ 0.0070, -0.0141, -0.0604,  ..., -0.0562,  0.0630, -0.0579],
        [ 0.1322,  0.1307, -0.1301,  ...,  0.1323, -0.2927,  0.1260],
        [-0.0311, -0.0939, -0.0670,  ...,  0.0050, -0.0416, -0.0016],
        ...,
        [-0.0549, -0.1435,  0.0342,  ..., -0.1831, -0.0141,  0.0053],
        [-0.0200,  0.1263,  0.0211,  ...,  0.0343, -0.0665, -0.0332],
        [ 0.0200, -0.0013, -0.0372,  ..., -0.0182,  0.0462,  0.0373]],
       dtype=torch.float64)
torch.Size([64, 300])
tensor([1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0,
        1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1])
False
False


In [19]:
 def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    for batch in iterator:
        optimizer.zero_grad()
        text_vector = batch['vector']
        # text_vector = batch['vector'].to(device)
        predictions = model(text_vector)
        
        y_pred_tag = torch.round(torch.sigmoid(predictions))
        
        # labels = batch['label'].to(device)
        labels = batch['label']
        labels = labels.float()
        labels = labels.reshape([ len(labels),1])
        loss = criterion(predictions, labels)
        
        f1 = f1_score(labels.detach().numpy(), y_pred_tag.detach().numpy())
        acc = calc_accuracy(y_pred_tag, labels)


        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator), f1

In [20]:
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    with torch.no_grad():
        for batch in iterator:
            text_vector = batch['vector']
            # text_vector = batch['vector'].to(device)
            predictions = model(text_vector)
            y_pred_tag = torch.round(torch.sigmoid(predictions))
            labels = batch['label']
            # labels = batch['label'].to(device)
            labels = labels.float()
            labels = labels.reshape([len(labels),1])
            loss = criterion(predictions, labels)
            f1 = f1_score(labels, y_pred_tag)
            conf_matrix = classification_report(labels, y_pred_tag)
            acc = calc_accuracy(y_pred_tag, labels)
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator), f1, conf_matrix

In [21]:
import time
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [22]:
def calc_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    # rounded_preds = torch.round(torch.sigmoid(preds))
    
    correct = (preds == y).float() #convert into float for division 
    # print(correct)
    acc = correct.sum() / len(correct)
    return acc

In [17]:
import torch.nn as nn
import torch.nn.functional as F

class FastText(nn.Module):
    def __init__(self, embedding_length, output_dim):        
        super().__init__()
        
        self.embedding_length = embedding_length
        
        self.output_dim = output_dim

        # self.embedding = nn.Embedding(self.vocab_size, self.embedding_length, padding_idx=pad_idx)
        # self.embedding.weight = nn.Parameter(word_embeddings, requires_grad = False)
        self.fc1 = nn.Linear(self.embedding_length, 512)
        self.fc2 = nn.Linear(512,256)
        self.fc3 = nn.Linear(256,128)
        self.fc4 = nn.Linear(128, output_dim)
        self.relu = nn.ReLU()
        self.dropout1 = nn.Dropout(p=0.1)
        self.dropout2 = nn.Dropout(p=0.4)
        
    def forward(self, text):        
        pooled_drop_op = self.dropout1(text)
        fc1_op = self.fc1(pooled_drop_op)
        fc1_drop_op = self.dropout2(fc1_op)
        fc2_op = self.fc2(fc1_drop_op)
        fc2_drop_op = self.dropout2(fc2_op) 
        fc3_op = self.fc3(fc2_drop_op)       
        return self.fc4(fc3_op)
        # text = self.relu(self.fc1(text))
        # text = self.relu(self.fc2(text))
        # text = self.relu(self.fc3(text))
        # text = self.dropout1(text)
        # return self.fc4(text)
        

In [18]:
EMBEDDING_DIM = 300
OUTPUT_DIM = 1
model = FastText( EMBEDDING_DIM,OUTPUT_DIM)
model = model.double()

In [19]:
import torch.optim as optim
optimizer = optim.Adam(model.parameters(), lr=1e-5)
criterion = nn.BCEWithLogitsLoss()
# criterion = criterion.to(device)

In [26]:
N_EPOCHS =15
best_valid_loss = float('inf')
for epoch in range(N_EPOCHS):
    start_time = time.time()
    train_loss, train_acc, train_f1 = train(model, train_dataloader, optimizer, criterion)
    valid_loss, valid_acc, val_f1, conf_matrix = evaluate(model, val_dataloader, criterion)
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% | Train F1:{train_f1}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% | Val F1:{val_f1}|\n{conf_matrix}')

/home/tjss/miniconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch: 01 | Epoch Time: 0m 13s
	Train Loss: 0.675 | Train Acc: 61.04% | Train F1:0.0
	 Val. Loss: 0.668 |  Val. Acc: 61.02% | Val F1:0.0|
              precision    recall  f1-score   support

         0.0       0.52      1.00      0.68        26
         1.0       0.00      0.00      0.00        24

    accuracy                           0.52        50
   macro avg       0.26      0.50      0.34        50
weighted avg       0.27      0.52      0.36        50

Epoch: 02 | Epoch Time: 0m 13s
	Train Loss: 0.662 | Train Acc: 61.05% | Train F1:0.0
	 Val. Loss: 0.658 |  Val. Acc: 61.04% | Val F1:0.0|
              precision    recall  f1-score   support

         0.0       0.60      1.00      0.75        30
         1.0       0.00      0.00      0.00        20

    accuracy                           0.60        50
   macro avg       0.30      0.50      0.37        50
weighted avg       0.36      0.60      0.45        50

Epoch: 03 | Epoch Time: 0m 13s
	Train Loss: 0.651 | Train Acc: 60.97% 

In [27]:
# 77% 0.68
test_loss, test_acc, test_f1, conf = evaluate(model, test_dataloader, criterion)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}% | Test F1: {test_f1}\n{conf}')

Test Loss: 0.518 | Test Acc: 74.85% | Test F1: 0.5161290322580646
              precision    recall  f1-score   support

         0.0       0.79      0.77      0.78        35
         1.0       0.50      0.53      0.52        15

    accuracy                           0.70        50
   macro avg       0.65      0.65      0.65        50
weighted avg       0.71      0.70      0.70        50



In [28]:
torch.save(model.state_dict(),prefix+'data/MeanEmbeddingsFasttext_NRCJoyVsNotJoy_wordLevelBackground(All Reviews)Shap.pt' )

In [20]:
model1 = FastText(EMBEDDING_DIM,OUTPUT_DIM)
model1.load_state_dict(torch.load(prefix+'data/MeanEmbeddingsFasttext_NRCJoyVsNotJoy_wordLevelBackground(All Reviews)Shap.pt'))
model1.eval()

FastText(
  (fc1): Linear(in_features=300, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=128, bias=True)
  (fc4): Linear(in_features=128, out_features=1, bias=True)
  (relu): ReLU()
  (dropout1): Dropout(p=0.1, inplace=False)
  (dropout2): Dropout(p=0.4, inplace=False)
)

In [21]:
%pip install shap
import shap

Note: you may need to restart the kernel to use updated packages.


In [22]:
test_df = pd.read_csv(prefix+"data/new_doc_level_train.csv")
test_df.head(5)

,Unnamed: 0,message,emotion_code
0,11390,"io domani volevo vestirmi tutta figa, ma il mi...",0
1,1136,"churchhhh! then exam stuff, and making 30 dang...",1
2,9805,empty fuel tank & a $ load of traffic,1
3,16383,he who cannot forgive others destroys the brid...,1
4,12353,hoje meu dia vai ser study study and study. te...,0


In [23]:
def get_word_embeddings(dataframe_csv):
  vocab={}
  bg = torch.empty((0,300))
  df = pd.read_csv(dataframe_csv)
  for idx, row in df.iterrows():
    curr_text = df.iloc[idx,1]
    curr_label = df.iloc[idx,2]

    for word in word_tokenize(curr_text):
      if(word in embeddings.keys() and word not in vocab.keys()):
        vocab[word]=0
        curr_embed = torch.from_numpy(np.array(embeddings[word]))
        curr_embed = curr_embed.reshape((1, len(curr_embed)))
        bg = torch.cat((bg, curr_embed))
#     if(idx==idx_limit):
#       break

  return bg, vocab


In [24]:
new_bg,_ = get_word_embeddings(prefix+"data/new_doc_level_train.csv" )
print(new_bg.size())

torch.Size([16408, 300])


In [25]:
e = shap.DeepExplainer(model, new_bg)

In [26]:
explain_examples, vocab1  = get_word_embeddings(prefix+"data/new_doc_level_test.csv")
print(explain_examples.size())

torch.Size([8820, 300])


In [27]:
st = time.time()
shap_vals = e.shap_values(explain_examples.float())
print(time.time()-st)

45068.27856755257


In [28]:
print(len(shap_vals))

8820


In [29]:
print(len(vocab1))

8820


In [30]:

print((shap_vals).shape)

(8820, 300)


In [31]:
shap_means_joy = shap_vals.mean(axis=1)

In [32]:
idx=0
for word in vocab1:
  vocab1[word]=shap_means_joy[idx]
  idx+=1

In [33]:
for word in vocab1:
  print(word, " ", vocab1[word])

ed   -2.6516840080106573e-06
balls   3.696317244902205e-05
says   4.666428184691311e-05
antiques   0.00012153247244784423
roadshow   0.000147979654958522
makes   -1.672606529310909e-05
him   -8.927623938701033e-07
cry   -6.353159073345462e-05
.   5.899292458683239e-05
a   2.545685644140554e-06
victory   -0.00013508900122546097
at   -8.14535395572822e-05
old   -5.3421160528150216e-05
trafford   -8.412405406408938e-05
is   3.397331451025235e-05
my   -9.823285153506265e-06
tipping   4.87305669058573e-05
point   1.225657462773346e-05
heel   1.1420149916925663e-05
iets   2.388956827483203e-05
hulp   -3.0765824788738136e-05
mag   3.548907164890846e-05
wel   5.2610981110222794e-05
toch   -1.102821590170343e-05
?   5.387180422526068e-05
#   -8.034630687177897e-05
apparently   -4.02138411462767e-05
10   -0.0001391698285182014
people   -0.00012251827539384215
every   -1.277094519096396e-05
year   2.844870005844632e-05
die   -7.412404061781748e-05
from   6.001309718098871e-05
vending   8.80709056

amazing   2.9124562209159646e-05
woke   -4.512327446586062e-05
different   2.4891736391785223e-05
set   2.6798956317116106e-05
clothes   -1.6902706777763392e-05
last   -0.00010843222033369071
isaiah   -7.180978326345194e-05
he   -2.7757816473483822e-05
said   7.33607790105644e-05
joy   -5.261619340715532e-05
draw   -7.862261970514875e-05
water   -6.350369397798286e-05
wells   -0.0001336823581600543
salvation   -3.416851314706417e-05
(   -2.2059777500747283e-05
niv   -4.165196727028743e-05
)   1.8082541954076828e-05
gisteren   7.997953059977855e-05
ontzettend   6.895980139532879e-05
leuke   5.635344252111929e-05
surprise   3.7463027139601917e-05
party   1.7472093838787477e-05
gehad   -1.421771929693627e-05
jarige   2.7902185973648416e-05
job   -3.581602979687138e-05
helemaal   0.00013147048531834575
blij   9.887948412749298e-05
finished   -2.0070280087606085e-05
birthday   7.541213788662004e-05
cake   6.865251973578575e-05
ice   6.56058403119377e-05
cream   0.00012080068709453049
happy 

looking   -5.7320240956439926e-06
government   -3.140182986303056e-05
finals   -7.344877367624708e-05
review   1.7749265292020667e-05
headache   5.8291198641556784e-05
weekend   -1.4629054740114422e-05
oo   1.9063989490893878e-05
ma   6.322909579148229e-05
wla   3.701859232470058e-05
shay   0.00014370291130445508
finally   -3.138313808791059e-05
get   1.1757204320826029e-06
huge   -2.2084683454895374e-05
cold   1.9579987746720736e-05
ill.   -6.944757469454521e-05
joys   -6.224105791369028e-05
england   7.31078892466292e-05
mention   -3.749241600685158e-05
walking   -6.072419553456356e-05
down   2.0263887035498472e-05
street   2.0998990783653456e-05
knee   0.0001049676669688067
gave   -4.913579019291053e-06
fell   8.603174219314513e-05
looked   2.8945612252179675e-05
haha   5.181284788481328e-05
tide   -9.94778623271436e-05
turning   -6.043656969333032e-05
may   1.8690129503736916e-05
comfort   -8.524526908636067e-05
food   4.972943571674206e-05
december   6.126758484867929e-05
monday  

fais   3.107624871001077e-05
impressionnant   4.117189277181935e-05
community   -7.779491464406905e-05
service   -0.00010776734229060822
straight   4.3927357674817533e-05
eye   -2.797610896685162e-05
nowhere   7.261028412976541e-05
area   -3.37828544932047e-05
they   -5.884504793894847e-05
drop   -0.00010454349897125154
surprised   3.650418477710875e-05
visit   -2.0472594519238713e-05
log   -3.9998708360991736e-05
would   7.447012388401489e-05
say   -1.5344322967454685e-05
lesson   -0.00013601096061050628
spent   -6.359495665400726e-06
dentists   -9.269042787269886e-06
organization   5.636092262728501e-05
49ers   -0.00011341916377544753
sum   0.0001068005366783271
frauds   2.6628079471131254e-05
nfc   -4.511160549829944e-05
afc   -6.599507542988187e-05
big   -7.141542875857826e-05
boys   -7.440125655495786e-05
deeply   -2.0118422288962763e-05
hear   -2.042911361802128e-05
pray   -0.00010543450581381762
ways   -7.774415042095841e-05
â   -0.00020655027164159602
”   2.910786905993136e-05


lettin   2.3770132072277093e-05
along   6.937054281162245e-05
w.   1.855070209332096e-05
inspring   -6.471735985998094e-05
keating   -0.00011866285737794281
motions   2.1654710398484097e-05
goal   9.10394040940567e-05
person   -8.560542430703974e-05
contact   4.966050388844684e-06
leaves   3.2346782368543884e-05
nurtured   -0.00012571417157105524
supported   1.3407669078733351e-05
omar   -8.361237966951677e-05
calm   -0.0001296392466563434
everything   -5.02730201632093e-05
context   1.614092716683222e-05
meant   -2.1321214299960665e-05
battle   -3.99082258998869e-05
words   5.07500961288069e-05
marcus   -7.156844638585243e-05
collins   5.6338854116421824e-05
better   1.9420106877875304e-05
cozy   -9.508296643429836e-06
omg   4.671412575614658e-05
jk   0.00013157568963593707
fail   -5.305624589408665e-05
friend   -2.672712216295053e-06
fl   -3.774419789371345e-05
showed   2.608412312488454e-06
door   -7.62272129963706e-05
step   -0.0001251798537514281
tried   -1.9847489869171592e-05
ti

...

gods   -3.314144073055596e-05
mistakes   -7.074259757108241e-05
rude   5.666579698560289e-05
shares   6.0480800616565696e-05
offering   4.9035717694859855e-05
strangers   -4.749288017951789e-05
3years   5.890426611700391e-05
one.   -7.203448056337212e-05
makem   -0.00012938150302967437
girlfriend   1.9040071062572108e-05
jew   -9.189425969624007e-05
mannn   0.0001518458410106652
funnny   9.6434284016952e-06
crosses   0.00011443933904760028
slob   -0.00013099127276363428
daisy   7.106940656365307e-05
perfume   1.899260445481741e-05
poke   3.840191974871315e-05
stopped   -4.756028467944115e-05
groupie   0.00023471440240872105
satisfaction   6.300080942552479e-06
recording   -0.00010768264051746843
skype   -1.4472871238573626e-06
convo   0.0001276715348031824
rolled   7.549140777459913e-05
tuck   6.76544580095791e-05
biological   -0.00011754169109163347
spoken   4.9920345722349216e-05
hottest   3.219978935918544e-05
application   5.9296356551108296e-05
backup   -5.7063946175524316e-05
jug

bouchons   -7.585759243710179e-05
bordelais   -7.465729580857357e-05
behalf   5.8780909184377256e-05
zeal   1.466843816428232e-05
jail   0.00020623413096233057
baru   -7.116289383361893e-05
cotton   0.00014466392758353774
10-2   -9.02878571341771e-05
practicing   1.0706548158455759e-05
wayyy   0.00019021219409130795
deliverance   -9.853548404884343e-06
skill   -1.4050264138850335e-05
gracefulness   6.090159810847116e-05
classy   3.9128922003366425e-05
interest   0.00013736049876720898
lectures   -1.6905638029986806e-05
rihanna   0.0001246369221900843
sampling   -9.157759564590075e-06
chase   0.00010184049773541957
saxon   -0.00014225852792771593
vinyl   0.00018445733737447968
vertrouwen   0.00010120362433696144
emmer   0.0002373426890451826
wag   0.00010669243408825089
kalimutan   -2.4216212245977687e-05
nyo   0.00012396392099118636
sakin   -3.276620605604114e-05
hehe   1.9077704434310692e-05
bugger   -9.786730467089617e-05
tablets   -7.875351527978604e-05
diddly   -7.685953433080308e-

5day   -6.567991985440919e-05
usa   1.3950052904788466e-05
cfb   9.398867050852737e-05
sweetness   2.7362489924346555e-05
voelt   -2.0869994911442998e-05
zich   -9.683332264207757e-06
bert   -5.834790058231946e-05
ernie   -0.00011526307669121661
stinks   8.680509065266885e-05
confined   1.3198633775015397e-06
scream   -9.288359540585011e-06
behaviour   -2.561197528010353e-05
opinion   0.00017281797242474353
now-a-days   0.00013694214336248598
kidnapping   -5.252980703223579e-05
trafficking   -4.00771157182761e-05
corners   9.575356173541109e-05
mattress   0.00011029033905162216
boo   -1.5698326953320366e-05
literature   0.00017536843290859598
rio   -0.00020635038389929622
q-tip   -2.7852337604202342e-05
sem   -2.1634693161574373e-05
grana   4.9833866809240225e-05
bh   5.3026776374505934e-05
bosom   -2.3927028540544615e-05
low   8.700537382412607e-05
disney   5.422212109215756e-05
fri   5.687569070342437e-05
unltd   0.0001988505713168837
user   -2.4444793510599346e-05
25.-   -7.01898132

holders   0.000107620989553761
ughhhhh   7.752367168363339e-05
survived   -0.00015814315589925978
boooooo   7.390893397334797e-05
hannaford   -1.6402416078789342e-05
allnighter   -1.8178963546996647e-05
abc   6.708841202958965e-05
bowel   -4.0500338841421076e-05
movements   -8.186236707388843e-05
anal   -8.868578312024588e-05
domestic   4.080776007260115e-06
drones   -3.406997514216828e-05
drone   1.4435032842388106e-05
attacking   -3.535949511920855e-05
ummm   -2.9814924403662967e-05
oi   -2.400439900826218e-05
b2b   0.00011543881803385359
parental   4.753254598220122e-05
efforts   -6.503113283431631e-05
mating   -3.638136960204973e-05
performed   -7.144133066378938e-07
'ye   0.00011132822265082845
srry   -4.152836226402821e-05
plead   -2.3768210794681345e-06
lipstick   9.077729966834538e-05
shameful   4.360839729395115e-05
vulgar   0.00015894642191265266
steeped   0.00014448537136905524
belt   4.7783075237124656e-05
flirt   0.00015030019343563756
breach   -0.0001228430635464902
quizz

former   -4.3815144376471444e-05
llegando   -0.00016955140174584857
justo   -5.5447559547833225e-05
lados   -3.5873615416048634e-05
pensando   -5.0191333924216194e-05
planes   -5.108189368396416e-05
noche   -8.967343131258025e-05
thousands   -0.00017644654377191055
turing   -3.743823731765956e-05
sensory   -0.00014220345735220855
insurance   5.436969832036598e-05
regulation   -5.771252911336446e-05
contacts   7.051961698081372e-05
viendo   -0.00018330582879265496
shakira   7.069888069537862e-05
crei   9.875509198940547e-05
saber   9.799693631970082e-05
todo   -0.00020948141511219204
clicking   -6.339317224763298e-05
followback   0.00014078336428629466
naw   5.376943868987504e-05
utter   7.448161931761951e-05
events   -7.661819285706431e-05
cadeautje   6.528303899050605e-05
tweep   4.097423088276544e-05
melike   9.663445621805804e-05
overprotected   -0.0001274187475345027
40   -6.303346127199909e-05
assaults   -6.653095493883889e-05
underage   6.863260721763224e-05
ratio   6.46727374874

In [34]:
df = pd.DataFrame(list(vocab1.items()),columns = ['column1','column2']) 

In [35]:
lexicon_df_joy = df.sort_values(by='column2')

In [36]:
lexicon_df_joy.head(50)

,column1,column2
7395,speakon,-0.000347
2980,âœ,-0.000331
6678,estaba,-0.000321
1798,gassed,-0.000319
3990,sangre,-0.000309
7973,carers,-0.000284
7457,cleanup,-0.000284
4869,drained,-0.000271
6151,retirer,-0.000267
6680,temprano,-0.000265


In [37]:
lexicon_df_joy.tail(50)

,column1,column2
7671,geluk,0.000219
5745,weed,0.000219
4045,advertisements,0.000219
1722,lame,0.000221
5793,statistics,0.000221
3292,proverbs,0.000222
5241,scarfs,0.000223
4783,polishes,0.000224
6497,ucas,0.000224
2178,aeropostale,0.000224


In [38]:
lexicon_df_joy.to_csv(prefix+"results/MeanEmbeddingsFasttext_NRCJoyVsNotJoy_wordLevelBackground(All Reviews)Shap.csv")
